In [14]:
from pynq import Overlay
from pynq import DefaultIP
import pynq
import numpy as np
import os
    
class CNNDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        self.data_in = pynq.allocate(shape=(90,), dtype=np.float32)
        self.raw_outputs = pynq.allocate(shape=(3,), dtype=np.float32)
        self.raw_cnn_outputs = pynq.allocate(shape=(130,), dtype=np.float32)
        
    bindto = ["xilinx.com:hls:cnn_action_detection:1.0"] 
    def inference(self, inp):
        self.data_in[:] = inp[:]
        self.register_map.data_in_1=self.data_in.physical_address
        self.register_map.function_select=0
        self.register_map.CTRL.AP_START=1
        while(self.register_map.CTRL.AP_DONE == 0):pass
    
    def getResult(self):
        self.register_map.function_select=1
        self.register_map.raw_output_1=self.raw_outputs.physical_address
        self.register_map.cnn_output_1=self.raw_cnn_outputs.physical_address
        self.register_map.CTRL.AP_START=1
        while(self.register_map.CTRL.AP_DONE == 0):pass
        return self.register_map.result_out
    
    def resetBuffer(self):
        self.register_map.function_select=2
        self.register_map.CTRL.AP_START=1
        while(self.register_map.CTRL.AP_DONE == 0):pass

In [15]:
overlay = Overlay("bitstream/cnn.bit")

In [16]:
myip = overlay.cnn_action_detection_0

In [17]:
myip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  function_select = Register(function_select=write-only),
  data_in_1 = Register(data_in=write-only),
  data_in_2 = Register(data_in=write-only),
  result_out = Register(result_out=0),
  result_out_ctrl = Register(result_out_ap_vld=0, RESERVED=0),
  data_required = Register(data_required=0),
  data_required_ctrl = Register(data_required_ap_vld=0, RESERVED=0),
  raw_output_1 = Register(raw_output=write-only),
  raw_output_2 = Register(raw_output=write-only),
  cnn_output_1 = Register(cnn_output=write-only),
  cnn_output_2 = Register(cnn_output=write-only)
}

In [13]:
myip.register_map.CTRL.AUTO_RESTART=0

In [23]:
import numpy as np
data = np.array([
-0.86743164, -0.36791992, -0.36694336, 0.43896484, -0.019042969, 0.045410156,
-0.86376953, -0.3708496, -0.3696289, 0.5566406, 0.055419922, 0.10449219,
-0.86743164, -0.37573242, -0.3684082, 0.57788086, 0.08911133, 0.18139648,
-0.88134766, -0.3864746, -0.359375, 0.72436523, -0.09838867, 0.37475586,
-0.8955078, -0.41577148, -0.35009766, 1.0307617, -0.48168945, 0.592041,
-0.9086914, -0.46972656, -0.3461914, 1.4931641, -0.8378906, 0.6201172,
-0.90844727, -0.53027344, -0.35839844, 1.9311523, -0.8354492, 0.19628906,
-0.89331055, -0.5686035, -0.37963867, 1.9648438, -0.5595703, -0.4128418,
-0.8696289, -0.58569336, -0.41186523, 1.7304688, 0.03564453, -0.73779297,
-0.8586426, -0.61621094, -0.42358398, 1.435791, 0.64990234, -0.84594727,
-0.91967773, -0.6550293, -0.38916016, 1.1259766, 1.3962402, -1.3425293,
-0.91625977, -0.70458984, -0.38134766, 1.3891602, 1.1875, -1.4143066,
-0.90771484, -0.7495117, -0.4038086, 1.4365234, 1.6547852, -1.8349609,
-0.8901367, -0.78808594, -0.43701172, 1.5644531, 1.784668, -2.0019531,
-0.890625, -0.80981445, -0.4638672, 1.5517578, 2.005371, -2.227539,
-0.8930664, -0.8203125, -0.48706055, 1.3757324, 2.104248, -2.4533691,
-0.8815918, -0.84716797, -0.51586914, 1.1518555, 1.8269043, -2.5290527,
-0.8713379, -0.90478516, -0.54663086, 1.2416992, 1.9414062, -2.6208496,
-0.8684082, -0.954834, -0.5710449, 1.6625977, 1.9172363, -2.6760254,
-0.8581543, -0.98046875, -0.58984375, 1.8686523, 1.793457, -2.6679688,
-0.8552246, -1.0134277, -0.59106445, 2.0100098, 1.6474609, -2.7189941,
-0.8469238, -1.0161133, -0.57666016, 2.1166992, 1.3737793, -2.8132324,
-0.8227539, -1.005127, -0.56225586, 1.7160645, 1.4069824, -3.0197754,
-0.7976074, -1.0366211, -0.5498047, 1.3071289, 1.4421387, -3.3659668,
-0.7619629, -1.0773926, -0.53759766, 1.4040527, 1.4501953, -3.5498047,
-0.72924805, -1.1000977, -0.51953125, 1.6767578, 1.4191895, -3.3374023,
-0.69921875, -1.1062012, -0.49169922, 1.824707, 1.213623, -3.159912,
-0.64746094, -1.0998535, -0.4633789, 1.7792969, 1.0805664, -3.2556152,
-0.58251953, -1.0854492, -0.43701172, 1.6818848, 1.1791992, -3.3493652,
-0.529541, -1.0759277, -0.40576172, 1.7253418, 1.244873, -3.270752,
-0.4724121, -1.0397949, -0.3671875, 1.7978516, 1.144043, -3.1142578,
-0.4074707, -0.9926758, -0.3239746, 1.7353516, 1.0456543, -2.9868164,
-0.33276367, -0.9572754, -0.28979492, 1.7067871, 0.95092773, -2.8063965,
-0.24560547, -0.91430664, -0.2614746, 1.7160645, 0.98095703, -2.6757812,
-0.16259766, -0.86938477, -0.22875977, 1.5651855, 1.0732422, -2.6193848,
-0.09814453, -0.82666016, -0.19628906, 1.496582, 1.0437012, -2.5263672,
-0.041259766, -0.7907715, -0.16503906, 1.4943848, 0.9448242, -2.317871,
0.017089844, -0.73950195, -0.13256836, 1.4748535, 0.8828125, -2.1401367,
0.067871094, -0.7043457, -0.099609375, 1.3461914, 0.86694336, -1.9645996,
0.111328125, -0.6645508, -0.064941406, 1.3317871, 0.80200195, -1.8652344,
0.1496582, -0.62646484, -0.028320312, 1.2387695, 0.7524414, -1.7390137,
0.18652344, -0.5878906, 0.014892578, 1.081543, 0.61376953, -1.7011719,
0.22851562, -0.5617676, 0.052734375, 0.8972168, 0.5258789, -1.6828613,
0.27563477, -0.5463867, 0.08203125, 0.82958984, 0.48608398, -1.6086426,
0.32006836, -0.53564453, 0.10839844, 0.90405273, 0.5349121, -1.4624023,
0.34301758, -0.5246582, 0.14501953, 0.90527344, 0.4736328, -1.3369141,
0.3774414, -0.51538086, 0.17773438, 0.91967773, 0.3425293, -1.3283691,
0.4128418, -0.5004883, 0.20654297, 0.84716797, 0.33422852, -1.340332,
0.44360352, -0.4880371, 0.22949219, 0.7541504, 0.36547852, -1.4130859,
0.4790039, -0.46166992, 0.2541504, 0.7446289, 0.41918945, -1.5041504,
0.50268555, -0.42138672, 0.28198242, 0.7011719, 0.41918945, -1.529541,
0.52734375, -0.37573242, 0.30688477, 0.66186523, 0.37036133, -1.515625,
0.55249023, -0.3251953, 0.3317871, 0.57250977, 0.34594727, -1.4470215,
0.57666016, -0.27246094, 0.36279297, 0.58251953, 0.30932617, -1.3044434,
0.6035156, -0.21411133, 0.38989258, 0.62524414, 0.18798828, -1.0959473,
0.63378906, -0.16455078, 0.41064453, 0.66918945, 0.17358398, -0.8034668,
0.65478516, -0.114990234, 0.43603516, 0.6555176, 0.19335938, -0.56884766,
0.67211914, -0.06542969, 0.46191406, 0.5517578, 0.14990234, -0.43041992,
0.685791, -0.03125, 0.48339844, 0.41723633, 0.07421875, -0.31713867,
0.69750977, -0.0073242188, 0.5031738, 0.31347656, 0.01586914, -0.25,
0.7089844, 0.010253906, 0.52075195, 0.19165039, -0.013183594, -0.19726562,
0.7207031, 0.016113281, 0.53271484, 0.0690918, -0.0056152344, -0.14501953,
0.7290039, 0.0056152344, 0.545166, 0.025390625, -0.0063476562, -0.053222656,
0.7351074, -0.013427734, 0.55566406, 0.022216797, -0.040283203, 0.017333984,
0.73876953, -0.03100586, 0.56689453, -0.031982422, -0.055664062, -0.01953125,
0.7402344, -0.053955078, 0.57421875, -0.14428711, -0.10913086, -0.12451172,
0.74853516, -0.080566406, 0.57373047, -0.15161133, -0.12915039, -0.1538086,
0.76123047, -0.103271484, 0.5695801, -0.09423828, -0.07421875, -0.13305664,
0.7680664, -0.11743164, 0.5686035, -0.035888672, -0.033691406, -0.06933594,
0.7717285, -0.12451172, 0.5673828, -0.036132812, -0.03466797, -0.08276367,
0.7753906, -0.1303711, 0.5629883, -0.063964844, -0.032958984, -0.11621094,
0.77734375, -0.13427734, 0.55981445, -0.03149414, -0.013427734, -0.068603516,
0.78027344, -0.13574219, 0.55566406, 0.02368164, -0.012207031, -0.017822266,
0.782959, -0.13256836, 0.5515137, 0.011962891, 0.00390625, -0.012451172,
0.7819824, -0.13085938, 0.5517578, -0.013183594, 0.037353516, 0.0014648438
], dtype = np.float32)

In [24]:
data.shape

(450,)

In [25]:
# sliding window with window size 15, stride 5
data_size = 75 * 6
window_size = 15
window_stride = 5
for j in range(0, len(data) - window_size*6 + 1, window_stride*6):
    print(data[j:j + 6*window_size])
    myip.inference(data[j:j + 6*window_size])
    print(f"result: {hex(myip.getResult())}")
    print(f"raw output: ")
    print(myip.raw_outputs)
    print(f"remaining data required: {int(myip.register_map.data_required)}")
    print("\n")
print(myip.raw_cnn_outputs)
myip.resetBuffer()

[-0.86743164 -0.36791992 -0.36694336  0.43896484 -0.01904297  0.04541016
 -0.86376953 -0.3708496  -0.3696289   0.5566406   0.05541992  0.10449219
 -0.86743164 -0.37573242 -0.3684082   0.57788086  0.08911133  0.18139648
 -0.88134766 -0.3864746  -0.359375    0.72436523 -0.09838867  0.37475586
 -0.8955078  -0.41577148 -0.35009766  1.0307617  -0.48168945  0.592041
 -0.9086914  -0.46972656 -0.3461914   1.4931641  -0.8378906   0.6201172
 -0.90844727 -0.53027344 -0.35839844  1.9311523  -0.8354492   0.19628906
 -0.89331055 -0.5686035  -0.37963867  1.9648438  -0.5595703  -0.4128418
 -0.8696289  -0.58569336 -0.41186523  1.7304688   0.03564453 -0.73779297
 -0.8586426  -0.61621094 -0.42358398  1.435791    0.64990234 -0.84594727
 -0.91967773 -0.6550293  -0.38916016  1.1259766   1.3962402  -1.3425293
 -0.91625977 -0.70458984 -0.38134766  1.3891602   1.1875     -1.4143066
 -0.90771484 -0.7495117  -0.4038086   1.4365234   1.6547852  -1.8349609
 -0.8901367  -0.78808594 -0.43701172  1.5644531   1.784668

In [19]:
myip.resetBuffer()

In [10]:
int(myip.getResult())

4294967295

In [23]:
hex(copy_npy(data[0:90]).device_address)

'0x5cca4000'